In [38]:
from importlib import reload
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor

from transformers import (
    AutoConfig, AutoModelForCausalLM, AutoTokenizer
)

pl.seed_everything(42)

Global seed set to 42


42

## tokenizer and dataset

In [3]:
name = "HooshvareLab/gpt2-fa"
tokenizer = AutoTokenizer.from_pretrained(name)

tokenizer.add_special_tokens({
    "bos_token": '</s>',
    "eos_token": '</s>', 
    "pad_token": '<pad>',
    "unk_token": '<unk>',
})

0

In [36]:
import src.data
reload(src.data)
from src.data import PoemDataset, get_dataloaders

dataset = PoemDataset(tokenizer, 'data/all_poems.json', max_len=512)
len(dataset)

11579631

In [43]:
tl, vl = get_dataloaders(dataset, val_frac=0.1, batch_size=4)

train dataset has 10421668 samples and val dataset has 1157963 samples


In [23]:
b = next(iter(tl))
for k in b:
    print(k, b[k].shape)

input_ids torch.Size([32, 186])
attention_mask torch.Size([32, 186])


## model

In [33]:
import src.model
reload(src.model)
from src.model import PoetFormer

model = PoetFormer(pretrained="HooshvareLab/gpt2-fa")

using pretrained model: HooshvareLab/gpt2-fa


In [35]:
model.count_parameters()

118099200

In [34]:
res = model.generate(num_return_sequences=1, max_length=128, n_beam=1)
for r in res:
    print(r)

Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.


حافظ:
جغند. از این بابت به من خیلی حق می‌دهند. من هم از این قاعده مستثنی نیستم. اما اگر کسی بود که می‌گفت تو آدم خیلی خفنی هستی که نمی‌تونی از پسش بربیای. چرا باید برای تو می‌جنگیم؟ آیا واقعا باید برای من جنگ باشه؟ چرا باید به من جنگی یاد بده؟ اگر به تو جنگ‌های زیادی یاد بدهم، آیا می‌توانم مثل آنها زندگی کنم؟ این دو سوال را در این متن با هم مرور می‌کنیم. خیلی از افراد که می‌خواهند در مورد این موضوع صحبت کنند، می‌پرسند که چرا من یک


## train

In [42]:
name = 'GPT2-fa-ganjoor-conditional'
print('model name:',name)

logger = TensorBoardLogger(save_dir='logs/', name=name)
lr_logger = LearningRateMonitor(logging_interval='step')
checkpoint = ModelCheckpoint(
    dirpath=f'weights/{name}/', 
    filename='{epoch}-{val_loss:.2f}', 
    monitor='val_loss',
    save_top_k=1, 
    period=1
)

trainer = pl.Trainer(
    benchmark=True, 
    gpus=0, 
    accumulate_grad_batches=1,
    logger=logger, 
    max_epochs=10,
    callbacks=[checkpoint, lr_logger]
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


model name: GPT2-fa-ganjoor-conditional


In [44]:
trainer.fit(model, tl, vl)


  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 118 M 
------------------------------------------
118 M     Trainable params
0         Non-trainable params
118 M     Total params
472.397   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: 0it [00:00, ?it/s]

Process Process-23:
Process Process-21:
Process Process-24:
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f54276311f0>
Traceback (most recent call last):
  File "/home/soroosh/projects/general_env/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/home/soroosh/projects/general_env/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.8/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/python3.8/multiprocessing/popen_fork.py", line 44, in wait
    if not wait([self.sentinel], timeout):
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/usr/lib/python3.8/selectors.py", line 415, in select
    fd_event_list = self._selector.poll(timeout)
KeyboardInterrupt: